In [307]:
import sympy as sp
import math

class Signal:
    def __init__(self, values = [], start = 0, finish = 0):
        self.values = values
        self.start = start
        self.finish = finish


    def __getitem__(self, n): 
        if self.start <= n <= self.finish:
            return self.values[n - self.start]
        else:
            raise IndexError("Неверный индекс")
    
    def __len__(self):
        return self.finish - self.start + 1

    def __add__(self, other):
        start = min(self.start, other.start)
        finish = max(self.finish, other.finish)
        
        result = []
        for i in range(start, finish + 1):
            res = 0
            if self.start <= i <= self.finish:
                res += self[i]
            if other.start <= i <= other.finish:
                res += other[i]
            result.append(res)

        return Signal(result, start, finish)




    def __mul__(self, other):
        result = [0 for i in range(len(self) + len(other) - 1)]

        for i in range(self.start, self.finish + 1):
            for j in range(other.start, other.finish + 1):
                result[i + j - self.start - other.start] += self[i] * other[j]
        return Signal(result, self.start + other.start, self.finish + other.finish)


    def compress(self, n:int):
        if self.start >= 0:
            start = (self.start+n-1) // n
        else:
            start = -(-self.start // n)
        if self.finish >= 0:
            finish = self.finish // n
        else:
            finish = -(-self.finish // n)

    
        #result = [self[i//n if i>=0 else -(-i//n)] for i in range(start, finish + 1)]
        if start >= finish and (self.start>start*n or self.finish<start*n):
            result = [0]
        else:
            result = [self[i*n] for i in range(start, finish + 1)]
        

        return Signal(result, start, finish)
    
    def stretch(self, n:int):
        start = self.start * n
        finish = self.finish * n

        result = [0 for i in range(start, finish + 1)]
        for i in range(self.start, self.finish + 1):
            result[i*n - start] += self[i]

        return Signal(result, start, finish)
    
    def __repr__(self):
        res = str(self.values) + '\n' + str(self.start) + ' ' + str(self.finish) 
        return res

    def shift(self, n):
        result = self
        result.start += n
        result.finish += n
        return result
    
    def norm(self, k):
        result = self
        result.values = [i/k for i in result.values]
        return result

    def remove_zeros(self):
        result = self

        while len(result)>1 and result[result.start] == 0:
            result.start += 1
            result.values.pop(0)

        while len(result)>1 and result[result.finish] == 0:
            result.finish -= 1
            result.values.pop(-1)

        return result
    
    def multiply(self, n):
        result = self
        result.values = [i*n for i in result.values]
        return result


    def haar(self):
        h0 = Signal([1/2**0.5, 1/2**0.5], 0, 1)
        h1 = Signal([1/2**0.5, -1/2**0.5], 0, 1)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)
        t0 = y0.stretch(2)
        t1 = y1.stretch(2)

        f0 = Signal([1/2**0.5, 1/2**0.5], -1, 0)
        f1 = Signal([-1/2**0.5, 1/2**0.5], -1, 0)

        v0 = t0 * f0
        v1 = t1 * f1

        s = v0 + v1

        return(s)

    def analysis_a(self):
        h0 = Signal([1], 0, 0)
        h1 = Signal([1/16, 0, -9/16, -16/16, -9/16, 0, 1/16], 0, 6)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis_a(self,other):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f0 = Signal([1/16, 0, -9/16, 16/16, -9/16, 0, 1/16], 0, 6)
        f1 = Signal([-1], 0, 0)

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1

        return(s)

    def analysis_b(self):
        h0 = Signal([1/2, -1/2], 0, 1)
        h1 = Signal([-1/8, 1/8, 8/8, 8/8, 1/8, -1/8], 0, 5)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis_b(self,other):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f0 = Signal([1/8, 1/8, -8/8, 8/8, -1/8, -1/8], 0, 5)
        f1 = Signal([1/2, 1/2], 0, 1)

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1

        return(s)

    def analysis_c(self):
        h0 = Signal([-1/4, 2/4, -1/4], 0, 2)
        h1 = Signal([-1/4, 2/4, 6/4, 2/4, -1/4], 0, 4)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis_c(self,other):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f0 = Signal([-1/4, -2/4, 6/4, -2/4, -1/4], 0, 4)
        f1 = Signal([1/4, 2/4, 1/4], 0, 2)

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1

        return(s)

    def analysis_d(self):
        h0 = Signal([-(2+3**0.5)/2, (1+3**0.5)/2, 1/2], 0, 2)
        h1 = Signal([-(2-3**0.5)/8, -(5-3**0.5)/8, -(3-3*3**0.5)/8, (1+3**0.5)/8, 1/8], 0, 4)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis_d(self,other):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f0 = Signal([-(2-3**0.5)/8, (5-3**0.5)/8, -(3-3*3**0.5)/8, -(1+3**0.5)/8, 1/8], 0, 4)
        f1 = Signal([(2+3**0.5)/2, (1+3**0.5)/2, -1/2], 0, 2)

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1

        return(s)

    def analysis_e(self):
        h0 = Signal([1/2, 3/2, -3/2, -1/2], 0, 3)
        h1 = Signal([1/8, 3/8, 3/8, 1/8], 0, 3)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis_e(self,other):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f0 = Signal([1/8, -3/8, 3/8, -1/8], 0, 3)
        f1 = Signal([-1/2, 3/2, 3/2, -1/2], 0, 3)

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1

        return(s)


    def analysis_f(self):
        #h0 = Signal([(1+3**0.5)/(4*2**0.5), (3+3**0.5)/(4*2**0.5), (3-3**0.5)/(4*2**0.5), (1-3**0.5)/(4*2**0.5)], 0, 3)
        #h1 = Signal([(1-3**0.5)/(4*2**0.5), (3-3**0.5)/(4*2**0.5), (3+3**0.5)/(4*2**0.5), (1+3**0.5)/(4*2**0.5)], 0, 3)
        h0 = Signal([-(1+3**0.5)/(4*2**0.5), (3+3**0.5)/(4*2**0.5), -(3-3**0.5)/(4*2**0.5), (1-3**0.5)/(4*2**0.5)], 0, 3)
        h1 = Signal([(1-3**0.5)/(4*2**0.5), (3-3**0.5)/(4*2**0.5), (3+3**0.5)/(4*2**0.5), (1+3**0.5)/(4*2**0.5)], 0, 3)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis_f(self,other):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f0 = Signal([(1-3**0.5)/(4*2**0.5), -(3-3**0.5)/(4*2**0.5), (3+3**0.5)/(4*2**0.5), -(1+3**0.5)/(4*2**0.5)], 0, 3)
        f1 = Signal([(1+3**0.5)/(4*2**0.5), (3+3**0.5)/(4*2**0.5), (3-3**0.5)/(4*2**0.5), (1-3**0.5)/(4*2**0.5)], 0, 3)

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1

        return(s)

    def analysis_g(self):
        h0 = Signal([-1/16, 4/16, -6/16, 4/16, -1/16], 0, 4)
        h1 = Signal([-1, 4, -1], 0, 2)
        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis_g(self,other):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f0 = Signal([-1, -4, -1], 0, 2)
        f1 = Signal([1/16, 4/16, 6/16, 4/16, 1/16], 0, 4)

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1

        return(s)
    

    def analysis(self, h0, f0):
        h1 = Signal()
        h1.start = f0.start
        h1.finish = f0.finish
        h1.values = [-((-1)**i)*f0[i] for i in range(h1.start, h1.finish + 1)]

        #h0.print()
        #h1.print()

        r0 = self * h0
        r1 = self * h1

        y0 = r0.compress(2)
        y1 = r1.compress(2)

        return(y0,y1)


    def synthesis(self,other, h0, f0, shift = 0, norm = 1):
        t0 = self.stretch(2)
        t1 = other.stretch(2)

        f1 = Signal()
        f1.start = h0.start
        f1.finish = h0.finish
        f1.values = [((-1)**i)*h0[i] for i in range(f1.start, f1.finish + 1)]

        #f0.print()
        #f1.print()

        v0 = t0 * f0
        v1 = t1 * f1

        #v0.print()
        #v1.print()

        s = v0 + v1
        s1 = s.shift(shift)
        s2 = s1.norm(norm)

        return(s2)
    
    def analysis_cascade(self, h0, f0, depth = -1):

        ans = []
        f = Signal()
        h = Signal()
        s = self

        max_depth = int(math.log2(len(self)))

        if depth == -1 or depth > max_depth:
            depth = max_depth

        #while len(s) > 2:
        for i in range(depth):
            f, h = s.analysis(h0, f0)
            ans.append(f.remove_zeros())
            s = h.remove_zeros()
            #print(s)
        ans.append(s)

        return ans
    

def synthesis_cascade(signal_list, h0, f0, shift = 0, norm = 1):

    s = signal_list[-1]
    s1 = Signal()

    for i in range(len(signal_list)-2, -1 , -1):
        s1 = signal_list[i].synthesis(s, h0, f0, shift, norm)
        #s2 = s1.shift(-3)
        s = s1.remove_zeros()
        print('*')
    
    return s

        


def factorization(poly):
    z = sp.symbols('z')
    simple_polys = []

    roots = sp.roots(sp.Poly(poly))

    if len(roots) > 0:
        for p in roots:
            for j in range(roots[p]):
                simple_polys.append(sp.Poly(z - p))

    else:
        roots = sp.Poly(poly).nroots()
        for p in roots:
            simple_polys.append(sp.Poly(z - p))

    print(simple_polys)

    ans = []
    for i in range(2**len(simple_polys)):
        f = 1
        g = 1
        for j in range(len(simple_polys)):
            if i & 2**j:
                f*=simple_polys[j]
            else:
                g*=simple_polys[j]
        
        s0 = Signal(sp.Poly(sp.simplify(f), z).all_coeffs(), 0, len(sp.Poly(sp.simplify(f), z).all_coeffs()) - 1)
        s1 = Signal(sp.Poly(sp.simplify(g), z).all_coeffs(), 0, len(sp.Poly(sp.simplify(g), z).all_coeffs()) - 1)

        ans.append([s0, s1])
    return ans











In [320]:
s1 = Signal([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 22], 1, 13)

#f0 = Signal([-1, 4, -1], 0, 2)
#h0 = Signal([1/16, 4/16, 6/16, 4/16, 1/16], 0, 4)

h0 = Signal([1, 1], 0, 1)
f0 = Signal([1, 1], 0, 1)

lst = s1.analysis_cascade(h0, f0, depth = 3)

for i in lst:
    print(i)

s = synthesis_cascade(lst, h0, f0, shift = -1, norm = 2)

print(s)


[3, 7, 11, 15, 19, 23, 22]
1 7
[-2, -2, -2, 22]
1 4
[22]
2 2
[-22]
2 2
*
*
*
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 22.0]
1 13


In [305]:
import wave
import numpy as np
wav = wave.open("test.wav", mode="r")
(nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()
content = wav.readframes(nframes)
types = {
    1: np.int8,
    2: np.int16,
    4: np.int32
}
samples = np.frombuffer(content, dtype=types[sampwidth])
print(len(samples))

channel = samples[0::nchannels]






5574656


In [328]:
#f0 = Signal([-1/2, 4/2, -1/2], 0, 2)
#h0 = Signal([1/8, 4/8, 6/8, 4/8, 1/8], 0, 4)


h0 = Signal([1/2**0.5, 1/2**0.5], 0, 1)
f0 = Signal([1/2**0.5, 1/2**0.5], 0, 1)



s = Signal(list(channel), 0, len(channel)-1)

lst = s.analysis_cascade(h0,f0, depth = 3)

for i in lst:
    print(i.values[:10])

lst[-4] = lst[-4].multiply(0)



s1 = synthesis_cascade(lst, h0, f0, shift = -1)

[np.float64(0.7071067811865475), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.7071067811865475), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
[np.float64(0.4999999999999999), np.float64(0.0), np.float64(-0.4999999999999999), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
[np.float64(0.3535533905932737), np.float64(-0.3535533905932737), np.float64(0.0), np.float64(0.0), np.float64(-0.7071067811865474), np.float64(0.0), np.float64(-0.3535533905932737), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
[np.float64(0.3535533905932737), np.float64(-0.3535533905932737), np.float64(0.0), np.float64(0.0), np.float64(0.7071067811865474), np.float64(-0.7071067811865474), np.float64(-0.3535533905932737), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
*
*
*


In [329]:
with wave.open("my_wave.wav", 'wb') as wf:

    wf.setnchannels(1)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)

    wf.writeframes(np.array([np.int16(i) for i in s1.values]).tobytes())

In [323]:
for i in range(len(lst)):
    with wave.open("my_wave"+str(i)+".wav", 'wb') as wf:

        wf.setnchannels(1)
        wf.setsampwidth(sampwidth)
        wf.setframerate(framerate)

        wf.writeframes(np.array([np.int16(i) for i in lst[i].values]).tobytes())




In [224]:
print(s.values[:10])

f0 = Signal([1, 2, 1], 0, 2)
h0 = Signal([-1, 2, 6, 2, -1], 0, 4)

s2, s3 = s.analysis(f0,h0)


s1 = s2.synthesis(s3,f0,h0,shift=-3,norm=16)
print([np.int16(i) for i in s1.values][:10])

[np.int16(-1), np.int16(-1), np.int16(-1), np.int16(-1), np.int16(0), np.int16(1), np.int16(2), np.int16(3), np.int16(4), np.int16(6)]


C:\Users\I_eli\AppData\Local\Temp\ipykernel_29228\1481607174.py:43: RuntimeWarning: overflow encountered in scalar add
  result[i + j - self.start - other.start] += self[i] * other[j]
C:\Users\I_eli\AppData\Local\Temp\ipykernel_29228\1481607174.py:43: RuntimeWarning: overflow encountered in scalar multiply
  result[i + j - self.start - other.start] += self[i] * other[j]
C:\Users\I_eli\AppData\Local\Temp\ipykernel_29228\1481607174.py:30: RuntimeWarning: overflow encountered in scalar add
  res += other[i]


[np.int16(0), np.int16(0), np.int16(0), np.int16(-1), np.int16(-1), np.int16(-1), np.int16(-1), np.int16(0), np.int16(1), np.int16(2)]


In [225]:
with wave.open("my_wave.wav", 'wb') as wf:

    wf.setnchannels(1)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)

    wf.writeframes(np.array([np.int16(i) for i in s1.values]).tobytes())


In [116]:
z = sp.symbols('z')
poly = z**6 - 9*z**4 - 16*z**3 - 9*z**2 + 1

ans = factorization(poly)

#print(ans)

f0 = ans[3][0]
h0 = ans[3][1]

print(f0)
print(h0)

s1 = Signal([1, 7, 3, 2, 4, 5], 0, 5)

lst = s1.analysis_cascade(h0, f0)

for i in lst:
    print(i)

s = synthesis_cascade(lst, h0, f0, shift = -3, norm = 16)

print(s)

[Poly(z + 1, z, domain='ZZ'), Poly(z + 1, z, domain='ZZ'), Poly(z + 1, z, domain='ZZ'), Poly(z + 1, z, domain='ZZ'), Poly(z + (sqrt(3)) - 2, z, sqrt(3), domain='ZZ'), Poly(z - (sqrt(3)) - 2, z, sqrt(3), domain='ZZ')]
[1, 2, 1]
0 2
[1, -2, -6, -2, 1]
0 4


KeyboardInterrupt: 

In [203]:
s1 = Signal([1, 7, 3, 2, 4, 5], 0, 5)

s2, s3 = s1.analysis_c()

#s2.print()
#s3.print()

s = s2.synthesis_c(s3)

print(s)


[0.0, 0.0, 0.0, 1.0, 7.0, 3.0, 2.0, 4.0, 5.0, 0.0, 0.0]
0 10


In [ ]:
s1 = Signal([1, 7, 3, 2, 4, 5], 0, 5)

s2, s3 = s1.analysis_c()

#s2.print()
#s3.print()

s = s2.synthesis_c(s3)

s.print()


[0.5, 0.5]
0 1


In [ ]:
import wave
import numpy as np
wav = wave.open("sample-9s.wav", mode="r")
(nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()
content = wav.readframes(nframes)
types = {
    1: np.int8,
    2: np.int16,
    4: np.int32
}
samples = np.frombuffer(content, dtype=types[sampwidth])
print(len(samples))

s = Signal()

with wave.open("my_wave.wav", 'wb') as wf:

    wf.setnchannels(nchannels)
    wf.setsampwidth(sampwidth)
    wf.setframerate(framerate)

    wf.writeframes(samples.tobytes())



845568


In [60]:
s1 = Signal([1, 7, 3, 2], 0, 3)

s2 = s1.haar()

s2.print()

[0.0, 0.9999999999999998, 6.999999999999997, 2.999999999999999, 1.9999999999999996, 0.0]
0 5


In [33]:
s = Signal ([1, 1, 1], -2, 0)
s1 = s.stretch(2)
s2 = s1.compress(2)
s1.print()
s2.print()

[1, 0, 1, 0, 1]
-4 0
[1, 1, 1]
-2 0


In [183]:
import sympy as sp
z = sp.symbols('z')
poly = z**6 - 9*z**4 - 16*z**3 - 9*z**2 + 1
#poly = z**2 + 2*z + 1
#poly = z**2 - 1
#factored_poly = sp.factor_list(poly, extension=sp.sqrt(3))
roots = sp.roots(sp.Poly(poly))


simple_polys = []
for p in roots:
    for j in range(roots[p]):
        simple_polys.append(sp.Poly(z - p))


ans = []
for i in range(2**len(simple_polys)):
    f = 1
    g = 1
    for j in range(len(simple_polys)):
        if i & 2**j:
            f*=simple_polys[j]
        else:
            g*=simple_polys[j]
    
    s0 = Signal(sp.Poly(sp.simplify(f), z).all_coeffs(), 0, len(sp.Poly(sp.simplify(f), z).all_coeffs()) - 1)
    s1 = Signal(sp.Poly(sp.simplify(g), z).all_coeffs(), 0, len(sp.Poly(sp.simplify(g), z).all_coeffs()) - 1)

    ans.append([s0, s1])


for p in ans:
    print(p)





[[1]
0 0, [1, 0, -9, -16, -9, 0, 1]
0 6]
[[1, 1]
0 1, [1, -1, -8, -8, -1, 1]
0 5]
[[1, 1]
0 1, [1, -1, -8, -8, -1, 1]
0 5]
[[1, 2, 1]
0 2, [1, -2, -6, -2, 1]
0 4]
[[1, 1]
0 1, [1, -1, -8, -8, -1, 1]
0 5]
[[1, 2, 1]
0 2, [1, -2, -6, -2, 1]
0 4]
[[1, 2, 1]
0 2, [1, -2, -6, -2, 1]
0 4]
[[1, 3, 3, 1]
0 3, [1, -3, -3, 1]
0 3]
[[1, 1]
0 1, [1, -1, -8, -8, -1, 1]
0 5]
[[1, 2, 1]
0 2, [1, -2, -6, -2, 1]
0 4]
[[1, 2, 1]
0 2, [1, -2, -6, -2, 1]
0 4]
[[1, 3, 3, 1]
0 3, [1, -3, -3, 1]
0 3]
[[1, 2, 1]
0 2, [1, -2, -6, -2, 1]
0 4]
[[1, 3, 3, 1]
0 3, [1, -3, -3, 1]
0 3]
[[1, 3, 3, 1]
0 3, [1, -3, -3, 1]
0 3]
[[1, 4, 6, 4, 1]
0 4, [1, -4, 1]
0 2]
[[1, -2 + sqrt(3)]
0 1, [1, 2 - sqrt(3), -4*sqrt(3) - 2, -6*sqrt(3) - 8, -7 - 4*sqrt(3), -2 - sqrt(3)]
0 5]
[[1, -1 + sqrt(3), -2 + sqrt(3)]
0 2, [1, 1 - sqrt(3), -3*sqrt(3) - 3, -3*sqrt(3) - 5, -2 - sqrt(3)]
0 4]
[[1, -1 + sqrt(3), -2 + sqrt(3)]
0 2, [1, 1 - sqrt(3), -3*sqrt(3) - 3, -3*sqrt(3) - 5, -2 - sqrt(3)]
0 4]
[[1, sqrt(3), -3 + 2*sqrt(3), -2 + sqrt(3

In [188]:
s1 = Signal([1, 7, 3, 2, 4, 5], 0, 5)

f0 = ans[0][0]
h0 = ans[0][1]

print(f0)
print(h0)

s2, s3 = s1.analysis(h0, f0)


s = s2.synthesis(s3, h0, f0)

print(s)

[1]
0 0
[1, 0, -9, -16, -9, 0, 1]
0 6
[0, 0, 0, -16, -112, -48, -32, -64, -80, 0, 0]
0 10
